In [1]:
import os
import torch
from torch import nn
import numpy as np
import random
from dataset import ModelnetDataset
from pathlib import Path
from torch.utils.data import DataLoader


from util import setup_seed, train, test, train_HHN, test_HHN, train_one4one, transform_angles, count_parameters, save_checkpoint
from model import One4All, HyperNetwork
from render import PointcloudRender
from tqdm import tqdm

from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.vis.plotly_vis import plot_scene
import plotly.graph_objects as go



In [2]:
setup_seed(666)
device_for_dataset = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
path = Path("./data/ModelNet10")
test_ds = ModelnetDataset(path, folder="test", device=device_for_dataset) # Just put dataset in mem or GPU
classes = {i: cat for cat, i in test_ds.classes.items()}

print('Test dataset size: ', len(test_ds))
print('Number of classes: ', len(test_ds.classes))

Preocessing test data in_mem = True and infor: Default infor


Prepare test dataset categories: 100%|██████████| 10/10 [00:06<00:00,  1.53it/s]

Test dataset size:  908
Number of classes:  10


In [4]:
from collections import Counter
# print(dict(Counter(sample['category'] for sample in train_ds.files)))

print(dict(Counter(sample['category'] for sample in test_ds.files)))


{'bathtub': 50, 'bed': 100, 'chair': 100, 'desk': 86, 'dresser': 86, 'monitor': 100, 'night_stand': 86, 'sofa': 100, 'table': 100, 'toilet': 100}


In [6]:
batch_size = 64

In [7]:
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=True)

: 

In [7]:
resume = './weight/LeNet5/E5950-bs256_lr0.002_6k-one4allcheckpoint.pth.tar'

gpu_computation = device_for_dataset
model = One4All() 
model = model.to(gpu_computation)
print(f'Num of model parameter: {count_parameters(model)}')

pr = PointcloudRender(              
                 img_size=128,
                 radius=0.0045,
                 points_per_pixel=40,
                 num_points = 4096)

checkpoint = torch.load(resume, map_location= gpu_computation)
model.load_state_dict(checkpoint['state_dict'])

loss_fn = nn.CrossEntropyLoss()


Num of model parameter: 138562


In [8]:

torch.cuda.empty_cache()

step = 30
acc = []
with tqdm(range(-180, 181, step), position=0) as t:
    for x in t:
        for y in tqdm(range(-180, 181, step), disable=True):
            for z in tqdm(range(-180, 181, step), disable=True):
                t.set_description('Test One4all')
                
                angles = torch.tensor([x,y,z])/180*torch.pi
                test_acc, test_loss = test(test_loader, model, loss_fn, pr, device=gpu_computation, angles=angles, verbose=False)
                acc.append(test_acc)
                t.set_postfix(Angles = [x,y,z],accuracy=test_acc) 

np.save('./one4all_acc.npy', np.array(acc))


Test One4all: 100%|██████████| 13/13 [52:23<00:00, 241.80s/it, Angles=[180, 180, 180], accuracy=0.485]  


In [9]:
one4all_acc = np.load('./accuracy_result_npy/one4all_acc.npy').mean()
print(f'{one4all_acc}')

0.5892997869427333
